# GPU Setting

In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
USE_GPU = True

if USE_GPU and tf.config.list_physical_devices('GPU'):
    device = 'GPU'
    print("Using GPU")
else:
    device = 'CPU'
    print("Using CPU")

# Loading Dataset

In [ ]:
import tarfile
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import random

# Path to the downloaded tgz file
tgz_path = "/home/asko/Documents/workspace/Fall-24/682/project/dataset/imagenette2.tgz"
extract_path = "./imagenette"  # Target folder for extraction

# Extract the file
with tarfile.open(tgz_path, "r:gz") as tar:
    tar.extractall(path=extract_path)
print("Extraction completed.")


transform = transforms.Compose([
    transforms.Resize((224,224)),  # Resize images to a size suitable for VGG16
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as per VGG16
])
extract_path_train = "./imagenette/imagenette2/train"
train_dataset = datasets.ImageFolder(
    root=extract_path_train,  # Imagenette URL
    transform=transform
)
extract_path_val = "./imagenette/imagenette2/val"
val_dataset = datasets.ImageFolder(
    root=extract_path_val,  # Imagenette URL
    transform=transform
)

batch_size = 8
sampler_train = sampler.SubsetRandomSampler(range(len(train_dataset)))
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler_train)

batch_size = 8
sampler_val = sampler.SubsetRandomSampler(range(1000))
val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_val)

# batch_size = 16
# sampler_test = sampler.SubsetRandomSampler(range(1000, len(val_dataset)))
# test_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler_test)


In [ ]:
import tensorflow as tf
from student.VGG_CMTKD_Student import VGG_CMTKD_Student
from student.VGG_CMTKD_Teacher import VGG_CMTKD_Teacher
from student.VGG_CMTKD_SuperTeacher import VGG_CMTKD_SuperTeacher
from Teacher_vgg.TeacherCNN import TeacherCNN
import torch

alpha = 0.5
beta  = 0.5
temperature = 3
pi1=0.6
pi2=0.4
bit_width = 2

# Load the VGG16 model
teacher_model = TeacherCNN(num_of_classes=10)
# teacher_1_model = VGG_CMTKD_Teacher(bit_width=bit_width, num_of_classes=10, teacher_idx=1)
# teacher_2_model = VGG_CMTKD_Teacher(bit_width=4, num_of_classes=10, teacher_idx=2)
# teacher_1_model = VGG_CMTKD_SuperTeacher(bit_width1=6, bit_width2=4, num_of_classes=10, pi1=pi1, pi2=pi2)

# student_model = VGG_CMTKD_Student(alpha=alpha, beta=beta, temperature=temperature, bit_width=2, pi1=pi1, pi2=pi2, num_of_classes=10)

device = 'cuda' if len(tf.config.list_physical_devices('GPU'))!=0 else 'cpu'
teacher_model.to(device)
# teacher_2_model.to(device)
# student_model.to(device)
print(device)



In [ ]:
def check_accuracy_part34(loader, model, model_name):
    print(f'Checking accuracy on validation set for {model_name}')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model((x, False))
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
            # del preds, scores, _
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))


In [ ]:
# import dill
# with open("./cache/TeacherFP/model_1.pkl", "rb") as f:
#     teacher_model = dill.load(f)

In [ ]:
import torch.optim as optim
import torch
import torch.nn.functional as F
import dill

teacher_1_optimizer = optim.Adam(teacher_model.parameters(), lr=0.01)
torch.set_grad_enabled(True)

num_epochs = 40
# batch_size = 16  
model_weights = None
print_every = 100

for epoch in range(num_epochs): 
    for batch_idx, (images, labels) in enumerate(train_loader):
        teacher_model.train()     
        # print('h1')
        images = images.to(device, dtype=torch.float32)
        labels = labels.to(device, dtype=torch.long)
        
        # print('h2')
        teacher_output = teacher_model((images, False))
        
        # for k,_ in teacher_1_model.cache.items():
        #     del teacher_1_model.cache[k]
        # torch.save(teacher_model.cache, f'cache/TeacherFP/cache_{epoch}_{batch_idx}.pth')
        # print(teacher_model.cache)
        # teacher_model.cache.clear()
        # teacher_1_model.cache.clear()
        loss = F.cross_entropy(input=teacher_output, target=labels)
        # print('h4')
        
        teacher_1_optimizer.zero_grad()
        loss.backward()
        teacher_1_optimizer.step()
            
        if batch_idx % print_every == 0:
            print('Iteration %d, loss = %.4f' % (batch_idx, loss.item()))
            check_accuracy_part34(val_loader, teacher_model, "teacher_1_model")
            print()
            
        torch.cuda.empty_cache()
        del teacher_output, images, labels, loss
    
    with open(f'cache/TeacherFP/model_{epoch}.pkl', "wb") as f:
        dill.dump(teacher_model, f)
        
    print(f"Epoch {epoch} complete")

In [ ]:
import dill
import torch
with open("./cache/TeacherFP/model_1", "rb") as f:
    teacher_model = dill.load(f)
device = 'cuda' if len(tf.config.list_physical_devices('GPU'))!=0 else 'cpu'
teacher_model.to(device)  
print(teacher_model.cache)
for batch_idx, (images, labels) in enumerate(train_loader):
    # print(len(teacher_model.cache))
    # teacher_model.train()
    
    images = images.to(device, dtype=torch.float32)
    output = teacher_model(images)
    print(teacher_model.cache)
    if batch_idx>2:
        break
    
    

In [ ]:
# import torch.optim as optim
# import torch

# student_optimizer = optim.Adam(student_model.parameters(), lr=0.01)
# teacher_1_optimizer = optim.Adam(teacher_1_model.parameters(), lr=0.01)
# teacher_2_optimizer = optim.Adam(teacher_2_model.parameters(), lr=0.01)
# # optimizer = optim.SGD(student_model.parameters(), lr=0.01, weight_decay=1e-4, momentum=0.9, nesterov=True)
# torch.set_grad_enabled(True)

# num_epochs = 40
# # batch_size = 16  
# model_weights = None
# print_every = 100

# for epoch in range(num_epochs): 
#     for batch_idx, (images, labels) in enumerate(train_loader):
#         teacher_1_model.train()
#         teacher_2_model.train()
#         student_model.train()           
#         # print('h1')
#         images = images.to(device, dtype=torch.float32)
#         labels = labels.to(device, dtype=torch.long)
#         # print('h2')
#         teacher_1_output = teacher_1_model(images)
#         teacher_2_output = teacher_2_model(images)
        
#         student_output = student_model(images)
#         # print('h3')
#         loss = student_model.loss(labels=labels, teacher_1_output=teacher_1_output, teacher_2_output=teacher_2_output, student_output=student_output)
#         # print('h4')
#         student_optimizer.zero_grad()
#         teacher_1_optimizer.zero_grad()
#         teacher_2_optimizer.zero_grad()
#         loss.backward()
#         student_optimizer.step()
#         teacher_1_optimizer.step()
#         teacher_2_optimizer.step()
            
#         if batch_idx % print_every == 0:
#             print('Iteration %d, loss = %.4f' % (batch_idx, loss.item()))
#             check_accuracy_part34(val_loader, teacher_1_model, "teacher_1_model")
#             check_accuracy_part34(val_loader, teacher_2_model, "teacher_2_model")
#             check_accuracy_part34(val_loader, student_model, "student_model")
#             print()
            
#         torch.cuda.empty_cache()
#         del teacher_1_output, teacher_2_output, student_output, images, labels, loss
        
#     print(f"Epoch {epoch} complete")

In [ ]:
# import dill
# with open("ta_model_retrained.pkl", "wb") as f:
#     dill.dump(ta_model, f)